### Create ETL Pipeline
#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# show total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Create Tables

#### Table For Query 1

##### The first query shall output the:
- artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

##### The first table has been designed with the following variables: 
- "(sessionid int, itemInSession int, artist_name text, song_title text, song_length float, PRIMARY KEY (sessionid, itemInSession))"

##### The PRIMARY key consists of both the 'sessionid' and 'iteminsession' variables, this is that so that:
- we can meet the requirement that all PRIMARY keys must be unique, as any of those two variables by themselves will not satify the unique requirement.
- we can meet the requirement to query the table by those two variables only within the SELECT WHERE statement.

In [8]:
# Query 1: Table Creation

query = "CREATE TABLE IF NOT EXISTS q1_playlist_sessionId338_itemInSession4"
query = query + "(sessionid int, itemInSession int, artist_name text, song_title text, song_length float, PRIMARY KEY (sessionid, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Query 1: Insert Data Into Table
file = 'event_datafile_new.csv'

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f, dialect='myDialect')
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO q1_playlist_sessionId338_itemInSession4 (sessionid, itemInSession, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
# Query 1: Query Table
query = "SELECT sessionid, artist_name, song_title, song_length FROM q1_playlist_sessionId338_itemInSession4 WHERE sessionid=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Create column names for dataframe and setup an empty list called 'data'
columns=('sessionid', 'artist_name', 'song_title', 'song_length') 
data=[] 

# Fill 'data' list with query results
for row in rows:
    #print (row.sessionid, row.artist_name, row.song_title, row.song_length) #retained for history
    values = [row.sessionid, row.artist_name, row.song_title, row.song_length] 
    data.append(dict(zip(columns, values)))
    
# Create dataframe and print
df = pd.DataFrame(data,columns=columns) 
df

,sessionid,artist_name,song_title,song_length
0,338,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Table For Query 2
##### The second query shall output the:
- name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### The second table has been designed with the following variables: 
- "(userid int, sessionid int, itemInSession int, artist_name text, song_title text, firstName text, lastName text, PRIMARY KEY ((userid, sessionid), itemInSession))"

##### The PRIMARY key consists of both the 'userid', 'sessionid' and 'iteminsession' variables, this is that so that:
- we can meet the requirement to query the table by the 'userid' and the 'sessionid' variables within the SELECT WHERE statement.
- we can meet the requirement to sort the song (sorted by the 'itemInSession' variable)
- we can meet the requirement that all PRIMARY keys must be unique. 

In [11]:
## Query 2: Table Creation

query = "CREATE TABLE IF NOT EXISTS q2_playlist_session182_userid10"
query = query + "(userid int, sessionid int, itemInSession int, artist_name text, song_title text, firstName text, lastName text, PRIMARY KEY ((userid, sessionid), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
## Query 2: Insert Data Into Table
file = 'event_datafile_new.csv'

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f, dialect='myDialect')
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO q2_playlist_session182_userid10 (userid, sessionid, itemInSession, artist_name, song_title, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
## Query 2: Query Table
query = "SELECT artist_name, song_title, firstname, lastname FROM q2_playlist_session182_userid10 WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Create column names for dataframe and setup an empty list called 'data'
columns=('artist_name', 'song_title', 'firstname', 'lastname') 
data=[] 
    
# Fill 'data' list with query results
for row in rows:
    #print (row.sessionid, row.artist_name, row.song_title, row.song_length)
    values = [row.artist_name, row.song_title, row.firstname, row.lastname] 
    data.append(dict(zip(columns, values)))
    
# Create dataframe and print
df = pd.DataFrame(data,columns=columns) 
df

,artist_name,song_title,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Table For Query 3
##### The third query shall output:
- every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### The second table has been designed with the following variables: 
- "(song_title text, userid int, firstName text, lastName text, PRIMARY KEY (song_title, userid))"

##### The PRIMARY key consists of both the 'song_title' and 'userid' variables, this is that so that:
- we can meet the requirement to query the table by the 'song_title' variable within the SELECT WHERE statement only.
- we can meet the requirement that all PRIMARY keys must be unique, specifically you need at addition of the 'userid' variable as there maybe a possibility of multiply users with the same first and last name that has listened to the subject song. Assuming the 'userid' is unique for each user this is a better variable to use in this query. 

In [14]:
## Query 3: Table Creation

query = "CREATE TABLE IF NOT EXISTS q3_userlist_AllHandsAgainstHisOwn"
query = query + "(song_title text, userid int, firstName text, lastName text, PRIMARY KEY (song_title, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
## Query 3: Insert Data Into Table
file = 'event_datafile_new.csv'

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f, dialect='myDialect')
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO q3_userlist_AllHandsAgainstHisOwn (song_title, userid, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
## Query 3: Query Table
query = "SELECT firstname, lastname FROM q3_userlist_AllHandsAgainstHisOwn WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Create column names for dataframe and setup an empty list called 'data'
columns=('firstname', 'lastname') 
data=[] 
    
# Fill 'data' list with query results
for row in rows:
    values = [row.firstname, row.lastname] 
    data.append(dict(zip(columns, values)))
    
# Create dataframe and print
df = pd.DataFrame(data,columns=columns) 
df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE IF EXISTS q1_playlist_sessionId338_itemInSession4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS q2_playlist_session182_userid10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS q3_userlist_AllHandsAgainstHisOwn"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()